In [1]:
import os
os.environ["MONGO_URI"] = "mongodb://localhost:27017"

# Setup the Jupyter version of Dash
from dash import Dash

# Dashboard component imports
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output

import plotly.express as px

# Data / plotting
import pandas as pd

# Import your CRUD module
from AnimalShelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################

# Connect to database via CRUD Module
# Your enhanced AnimalShelter class does NOT take username/password now
db = AnimalShelter()

# Pull all records
df = pd.DataFrame.from_records(db.read({}))

# Drop _id if it exists (prevents ObjectId issues in Dash DataTable)
df.drop(columns=["_id"], inplace=True, errors="ignore")


#########################
# Dashboard Layout / View
#########################

app = Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.H4("Dashboard by Zeb Berry IV")
    ]),

    html.Center(html.B(html.H1("CS-340 Dashboard"))),
    html.Hr(),

    html.Div([
        dcc.RadioItems(
            id="filter-type",
            options=[
                {"label": "Water Rescue", "value": "water"},
                {"label": "Mountain/Wilderness Rescue", "value": "mountain"},
                {"label": "Disaster/Tracking", "value": "disaster"},
                {"label": "Reset", "value": "reset"},
            ],
            value="reset",
            labelStyle={"display": "block"},
        )
    ]),
    html.Hr(),

    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict("records"),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[],
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left", "padding": "6px", "whiteSpace": "normal"},
    ),

    html.Br(),
    html.Hr(),

    html.Div(
        className="row",
        style={"display": "flex"},
        children=[
            html.Div(id="graph-id", className="col s12 m6"),
            html.Div(id="map-id", className="col s12 m6"),
        ],
    ),
])


#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output("datatable-id", "data"),
    [Input("filter-type", "value")]
)
def update_dashboard(filter_type):
    """
    Replaces the original db.filter_* methods by using db.read(query) directly.
    If your dataset uses different field names, update these queries to match.
    """

    # Default is "all"
    query = {}

    if filter_type == "water":
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
        }

    elif filter_type == "mountain":
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
        }

    elif filter_type == "disaster":
        query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
        }

    data = list(db.read(query))

    # Remove _id if present so Dash DataTable does not choke on ObjectId
    for item in data:
        item.pop("_id", None)

    return data


@app.callback(
    Output("graph-id", "children"),
    [Input("datatable-id", "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        dff = df
    else:
        dff = pd.DataFrame.from_dict(viewData)

    # If breed is missing, show a friendly message instead of crashing
    if "breed" not in dff.columns:
        return [html.Div("No 'breed' column found, so the pie chart cannot render.")]

    return [
        dcc.Graph(
            figure=px.pie(dff, names="breed", title="Animal Breed Distribution")
        )
    ]


@app.callback(
    Output("datatable-id", "style_data_conditional"),
    [Input("datatable-id", "selected_columns")]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        "if": {"column_id": i},
        "background_color": "#D2F3FF"
    } for i in selected_columns]


@app.callback(
    Output("map-id", "children"),
    [Input("datatable-id", "derived_virtual_data"),
     Input("datatable-id", "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or len(viewData) == 0:
        return html.Div("No data available for the map yet.")

    if index is None or len(index) == 0:
        return html.Div("Select a row to see the location on the map.")

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]

    # Try common lat/long column names first
    possible_lat_cols = ["location_lat", "lat", "Latitude", "latitude"]
    possible_lon_cols = ["location_long", "lon", "Longitude", "longitude", "long"]

    lat_col = next((c for c in possible_lat_cols if c in dff.columns), None)
    lon_col = next((c for c in possible_lon_cols if c in dff.columns), None)

    # If we can't find named columns, try the original column index approach safely
    lat_val = None
    lon_val = None

    if lat_col and lon_col:
        lat_val = dff.loc[row, lat_col]
        lon_val = dff.loc[row, lon_col]
    else:
        # Fall back to original assumption (13,14) only if dataframe is wide enough
        if dff.shape[1] > 14:
            lat_val = dff.iloc[row, 13]
            lon_val = dff.iloc[row, 14]

    if lat_val is None or lon_val is None:
        return html.Div("Could not find latitude/longitude columns for the map.")

    # Breed/name display fields (safe)
    breed_val = dff.loc[row, "breed"] if "breed" in dff.columns else "Unknown breed"
    name_val = dff.loc[row, "name"] if "name" in dff.columns else "Unknown"

    return dl.Map(
        style={"width": "1000px", "height": "500px"},
        center=[30.75, -97.48],
        zoom=10,
        children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(
                position=[lat_val, lon_val],
                children=[
                    dl.Tooltip(str(breed_val)),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(str(name_val))
                    ])
                ]
            )
        ],
    )


app.run(debug=True)



INFO:AnimalShelter:Connected to MongoDB database 'AAC' and collection 'animals'
